# Creating custom accessors

## Introduction

An accessor is a way of attaching a custom function to xarray types so that it can be called as if it were a method while retaining a clear separation between "core" xarray API and custom API. It enables you to easily extend and customize xarray's functionality while limiting naming conflicts and minimizing the chances of your code breaking with xarray upgrades.

If you've used [rioxarray](https://corteva.github.io/rioxarray/stable/) (e.g. `da.rio.crs`) or [hvplot](https://hvplot.holoviz.org/) (e.g. `ds.hvplot()`), you may have already used an xarray accessor without knowing it!

## Easy steps to create your own accessor

1. Create your custom class, including the mandatory `__init__` method
2. Add the `xr.register_dataarray_accessor()` or `xr.register_dataset_accessor()` 
3. Use your custom functions 

## Example 1: accessing scipy functionality

For example, imagine you're a statistician who regularly uses a special `skewness` function which acts on dataarrays but is only of interest to people in your specific field.

You can create a method which applies this skewness function to an xarray objects, and then register the method under a custom `stats` accessor like this

In [ ]:
from scipy.stats import skew


@xr.register_dataarray_accessor("stats")
class StatsAccessor:
    def __init__(self, da):
        self._da = da

    def skewness(self, dim):
        return self._da.reduce(func=skew, dim=dim)

Now we can conveniently access this functionality via the `stats` accessor

In [ ]:
ds['air'].stats.skewness(dim="time")

Notice how the presence of `.stats` clearly differentiates our new "accessor method" from core xarray methods.

## Example 2: creating your own functions